In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
print('Load bag bag_of_words_top_1000.feather')
bag_of_words_top_1000 = pd.read_feather('../../data/transform/bag_of_words_top_1000.feather')

In [3]:
def corpus_topics_top_words(model, features, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict[topic_idx] = [features[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
    return topic_dict

def song_topics(model, song):
    topic_dict = []
    for topic_idx, topic in enumerate(model.components_):
        topic_dict.append(sum(topic*song))
    return topic_dict

In [10]:
print('Identify topics within model')
from sklearn.decomposition import LatentDirichletAllocation

mxm_25_top1000 = LatentDirichletAllocation(n_topics=25, random_state=0)
mxm_25_top1000.fit(bag_of_words_top_1000.iloc[:,16:])

/Users/spare/anaconda3/envs/py3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/spare/anaconda3/envs/py3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=25,
             perp_tol=0.1, random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [13]:
print('Identify per/song topic weights')
top_per_topic_words = corpus_topics_top_words(mxm_25_top1000,bag_of_words_top_1000.iloc[:,16:].columns.values, 10)

#save per/song topic results to df
song_topic_weights = np.zeros([len(bag_of_words_top_1000.iloc[:,16:]),25])
temp = bag_of_words_top_1000.iloc[:,16:]
for i in tqdm(range(len(bag_of_words_top_1000))):
    song_weights = pd.Series(song_topics(mxm_25_top1000, temp.iloc[i]))
    song_topic_weights[i] = song_weights

100%|██████████| 237662/237662 [19:12<00:00, 206.19it/s]


In [17]:
initial_topic_names = list(range(25))
song_topic_weights_df = pd.DataFrame(data =song_topic_weights, columns=initial_topic_names)
song_topic_weights_df['track_id'] = bag_of_words_top_1000['track_id_']
df_topic_weights_reduced_df = song_topic_weights_df[['track_id',11,15,20]]
df_topic_weights_reduced_df.columns = 'track_id', 'love','death','religion'

In [18]:
#Top words in each topic
topic_words = {}
topic_words['love'] = top_per_topic_words[11]
topic_words['death'] = top_per_topic_words[15]
topic_words['religion'] = top_per_topic_words[20]
print(topic_words)

{'love': ['love', 'heart', 'sweet', 'true', 'give', 'enough', 'darling', 'touch', 'vision', 'found'], 'death': ['life', 'die', 'run', 'dead', 'kill', 'dream', 'blood', 'death', 'scream', 'clear'], 'religion': ['us', 'god', 'live', 'dance', 'people', 'heaven', 'hand', 'stand', 'angel', 'beautiful']}


In [19]:
bag_of_words_top_1000['love'] = df_topic_weights_reduced_df['love']
bag_of_words_top_1000['religion'] = df_topic_weights_reduced_df['religion']
bag_of_words_top_1000['death'] = df_topic_weights_reduced_df['death']

# Transform

In [24]:
original_top_1000 = pd.read_feather('../../data/transform/bag_of_words_top_1000.feather')

In [28]:
bag_of_words_top_1000_topics = bag_of_words_top_1000

In [29]:
#scale by word count
bag_of_words_top_1000_topics['sum'] = original_top_1000.iloc[:,16:].sum(axis=1)
bag_of_words_top_1000_topics['love'] = bag_of_words_top_1000_topics['love']/bag_of_words_top_1000_topics['sum']
bag_of_words_top_1000_topics['religion'] = bag_of_words_top_1000_topics['religion']/bag_of_words_top_1000_topics['sum']
bag_of_words_top_1000_topics['death'] = bag_of_words_top_1000_topics['death']/bag_of_words_top_1000_topics['sum']
bag_of_words_top_1000_topics.drop('sum', axis=1, inplace=True)

In [30]:
#rank and scale between 0 and 1
rank_order = np.array(range(len(bag_of_words_top_1000_topics)))/(len(bag_of_words_top_1000_topics)-1)

bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.sort_values(by='religion')
bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.reset_index(drop=True)
bag_of_words_top_1000_topics['religion'] = rank_order

bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.sort_values(by='death')
bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.reset_index(drop=True)
bag_of_words_top_1000_topics['death'] = rank_order

bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.sort_values(by='love')
bag_of_words_top_1000_topics = bag_of_words_top_1000_topics.reset_index(drop=True)
bag_of_words_top_1000_topics['love'] = rank_order

In [ ]:
#bag_of_words_top_1000_topics.to_feather('../../data/clean/bag_of_words_top_1000_weightedtopics.feather')

In [31]:
#yearly topics
print('Identify topic weights each year')
bag_with_year = bag_of_words_top_1000_topics[bag_of_words_top_1000_topics['release_year_'].notnull()].reset_index()

weights_yearly_clean = {}
weights_yearly_clean['song_count'] = [0] * len(bag_with_year['release_year_'].unique())
weights_yearly_clean['love'] = [0] * len(bag_with_year['release_year_'].unique())
weights_yearly_clean['death'] = [0] * len(bag_with_year['release_year_'].unique())
weights_yearly_clean['religion'] = [0] * len(bag_with_year['release_year_'].unique())
weights_yearly_clean['year'] = [0] * len(bag_with_year['release_year_'].unique())

for i, val in enumerate(bag_with_year['release_year_'].unique()):
    temp = bag_with_year[bag_with_year['release_year_']==val]
    weights_yearly_clean['song_count'][i] = len(temp)
    weights_yearly_clean['year'][i] = val
    weights_yearly_clean['love'][i] = temp.love.mean()
    weights_yearly_clean['death'][i] = temp.death.mean()
    weights_yearly_clean['religion'][i] = temp.religion.mean()

topics_per_year = pd.DataFrame(weights_yearly_clean).sort_values(by='year').reset_index(drop=True)

In [54]:
#artist weights
print('Identify topic weights for each artist (~21k iterations)')
weights_artist_clean = {}
weights_artist_clean['love'] = [0] * len(bag_of_words_top_1000_topics.artist_id_.unique())
weights_artist_clean['death'] = [0] * len(bag_of_words_top_1000_topics.artist_id_.unique())
weights_artist_clean['religion'] = [0] * len(bag_of_words_top_1000_topics.artist_id_.unique())
weights_artist_clean['artist_id'] = [0] * len(bag_of_words_top_1000_topics.artist_id_.unique())
weights_artist_clean['artist_name'] = [0] * len(bag_of_words_top_1000_topics.artist_id_.unique())
weights_artist_clean['longitude'] = [0] * len(bag_of_words_top_1000_topics.artist_id_.unique())
weights_artist_clean['latitude'] = [0] * len(bag_of_words_top_1000_topics.artist_id_.unique())
weights_artist_clean['song_count'] = [0] * len(bag_of_words_top_1000_topics.artist_id_.unique())


for i, val in tqdm(enumerate(bag_of_words_top_1000_topics.artist_id_.unique())):
    temp = bag_of_words_top_1000_topics[bag_of_words_top_1000_topics.artist_id_==val]
    weights_artist_clean['artist_id'][i] = val
    weights_artist_clean['artist_name'][i] = temp.artist_name_.iloc[0]
    weights_artist_clean['longitude'][i] = temp.longitude_.iloc[0]
    weights_artist_clean['latitude'][i] = temp.latitude_.iloc[0]
    weights_artist_clean['love'][i] = temp.love.mean()
    weights_artist_clean['death'][i] = temp.death.mean()
    weights_artist_clean['religion'][i] = temp.religion.mean()
    weights_artist_clean['song_count'][i] = len(temp)

topics_per_artist = pd.DataFrame(weights_artist_clean).sort_values(by='artist_name').reset_index(drop=True)

Identify topic weights for each artist ()


20116it [18:19, 18.29it/s]


In [57]:
topics_per_artist.to_feather('../../data/transform/artist_topic_weights.feather')
topics_per_year.to_feather('../../data/transform/year_topic_weights.feather')